In [25]:
# !pip install C:\Users\archi\Downloads\TA_Lib-0.4.28-cp310-cp310-win_amd64.whl
# !pip install pyyaml
# !pip install psutil

In [26]:
import os
import sys
import warnings
import gc
import re
from pathlib import Path

# 設置專案路徑
project_path = r'C:\Users\archi\Python\Project\tw_stock_analysis'
sys.path.append(project_path)

# 日期和時間
import time
from datetime import datetime

# 數據處理和分析
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
import talib

# 並行處理
from concurrent.futures import ThreadPoolExecutor, as_completed

# 類型提示
from typing import Dict, List, Tuple, Optional, Any

# 數據類
from dataclasses import dataclass, field

# 工具函數
import logging
import traceback
import shutil
import json
import psutil
from tqdm import tqdm

# 關閉警告
warnings.filterwarnings('ignore')

In [27]:
@dataclass
class TestFeatureConfig:
    """台股技術分析特徵生成器配置類別
    
    管理所有與特徵生成相關的配置參數，包括:
    - 基礎路徑設定
    - 測試資料設定
    - 產業分析參數
    - 特徵計算參數
    """
    
    # 基礎路徑設定
    BASE_DIR: str = "D:/Min/Python/Project/FA_Data"
    META_DATA_DIR: str = "meta_data"
    TEST_DATA_DIR: str = "test_data"
    LOG_DIR: str = "logs"
    FEATURES_DIR: str = "features"
    INDUSTRY_ANALYSIS_DIR: str = "industry_analysis"
    INDUSTRY_CORRELATION_DIR: str = "industry_correlation"

    # 私有記錄器實例
    _logger: Optional[logging.Logger] = None
    
    # 測試資料設定
    TEST_SETTING: Dict = field(default_factory=lambda: {
        'test_stocks': ['2330', '2317', '1101', '2891', '2303'],
        'start_date': '2024-01-01',
        'end_date': '2024-03-31'
    })
    
    # 產業分析參數
    INDUSTRY_PARAMS: Dict = field(default_factory=lambda: {
        'analysis_start_date': '2023-01-01',
        'analysis_end_date': '2024-12-31',
        'min_data_days': 30,
        'update_frequency': 'daily'
    })
    
    # 特徵計算參數
    FEATURE_PARAMS: Dict = field(default_factory=lambda: {
        'volume': {
            'short_period': 5,
            'long_period': 20,
            'min_volume': 1000
        },
        'volatility': {
            'short_period': 5,
            'long_period': 20,
            'std_window': 20
        },
        'trend': {
            'ma_period': 20,
            'momentum_window': 10
        },
        'technical': {
            'rsi_period': 14,
            'macd_params': {
                'fast_period': 12,
                'slow_period': 26,
                'signal_period': 9
            },
            'kd_params': {
                'fastk_period': 9,
                'slowk_period': 3,
                'slowd_period': 3
            },
            'bollinger_params': {
                'window': 20,
                'num_std': 2
            }
        }
    })
    
    def __post_init__(self):
        """初始化後設定"""
        # 設定基礎路徑
        self._setup_paths()
        # 初始化記錄器
        if self._logger is None:
            self._logger = logging.getLogger('TestFeatureConfig')
            self._logger.setLevel(logging.INFO)
        # 初始化記錄器
        self._initialize_logging()
        
    def _setup_paths(self):
        """設定並建立必要的目錄結構"""
        # 設定基礎路徑
        self.base_path = Path(self.BASE_DIR)
        self.meta_data_path = self.base_path / self.META_DATA_DIR
        self.test_data_path = self.base_path / self.TEST_DATA_DIR
        self.log_path = self.base_path / self.LOG_DIR
        self.features_path = self.base_path / self.FEATURES_DIR
        self.industry_analysis_path = self.base_path / self.INDUSTRY_ANALYSIS_DIR
        self.industry_correlation_path = self.base_path / self.INDUSTRY_CORRELATION_DIR
        
        # 確保所需目錄存在
        self._create_directories()
        
    def get_stock_data_path(self) -> Path:
        """取得股票數據檔案路徑"""
        return self.test_data_path / 'test_stock_data.csv'
        
    def get_test_tech_data(self) -> Path:
        """取得技術指標檔案路徑"""
        return self.features_path / 'test_tech_data.csv'
        
    def get_log_path(self) -> Path:
        """取得日誌檔案路徑"""
        return self.log_path / f'feature_generator_{datetime.now():%Y%m%d}.log'

    def validate_config(self) -> bool:
        """驗證配置的有效性"""
        try:
            # 1. 驗證路徑設定
            required_paths = [
                self.BASE_DIR,
                self.META_DATA_DIR,
                self.TEST_DATA_DIR,
                self.LOG_DIR,
                self.FEATURES_DIR
            ]
            
            for path in required_paths:
                if not path:
                    self.logger.error(f"路徑設定不能為空: {path}")
                    return False
                    
            # 2. 檢查日期範圍參數
            test_setting = self.TEST_SETTING
            try:
                start_date = pd.to_datetime(test_setting['start_date'])
                end_date = pd.to_datetime(test_setting['end_date'])
                
                if start_date >= end_date:
                    self.logger.error("起始日期必須早於結束日期")
                    return False
                    
                # 檢查日期範圍是否合理
                date_range = (end_date - start_date).days
                if date_range < 30:
                    self.logger.warning(f"測試日期範圍可能過短: {date_range}天")
                    
            except ValueError:
                self.logger.error("日期格式無效")
                return False
                
            # 3. 驗證特徵參數
            feature_params = self.FEATURE_PARAMS
            
            # 檢查移動平均期間
            ma_periods = [
                feature_params['volume']['short_period'],
                feature_params['volume']['long_period'],
                feature_params['volatility']['short_period'],
                feature_params['volatility']['long_period']
            ]
            
            if any(period <= 0 for period in ma_periods):
                self.logger.error("移動平均期間必須大於0")
                return False
                
            # 檢查技術指標參數
            tech_params = feature_params['technical']
            if tech_params['rsi_period'] <= 0:
                self.logger.error("RSI期間必須大於0")
                return False
                
            # 4. 驗證產業分析參數
            industry_params = self.INDUSTRY_PARAMS
            if industry_params['min_data_days'] < 20:
                self.logger.warning("產業分析的最小數據天數建議不小於20天")
                
            # 檢查更新頻率設定
            valid_frequencies = ['daily', 'weekly', 'monthly']
            if industry_params['update_frequency'] not in valid_frequencies:
                self.logger.error(f"無效的更新頻率: {industry_params['update_frequency']}")
                return False
                
            return True
            
        except Exception as e:
            self.logger.error(f"配置驗證失敗: {str(e)}")
            return False

    def _create_directories(self):
        """建立必要的目錄"""
        directories = [
            self.test_data_path,
            self.log_path,
            self.features_path,
            self.industry_correlation_path,
            self.industry_correlation_path / "weekly",
            self.industry_correlation_path / "monthly"
        ]
        
        for directory in directories:
            directory.mkdir(parents=True, exist_ok=True)
    
    def _initialize_logging(self):
        """初始化記錄系統"""
        if self._logger is None:
            self._logger = logging.getLogger('TestFeatureGenerator')
            self._logger.setLevel(logging.INFO)
            
            # 避免重複添加處理器
            if not self._logger.handlers:
                # 檔案處理器
                log_file = self.get_log_path()
                fh = logging.FileHandler(log_file, encoding='utf-8')
                fh.setFormatter(logging.Formatter(
                    '%(asctime)s [%(levelname)s] %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S'
                ))
                
                # 控制台處理器
                ch = logging.StreamHandler()
                ch.setFormatter(logging.Formatter('%(levelname)s: %(message)s'))
                
                self._logger.addHandler(fh)
                self._logger.addHandler(ch)
    
    @property
    def logger(self):
        """取得記錄器實例"""
        return self._logger

In [28]:
class DataValidator:
    """資料驗證器類別"""
    
    def __init__(self, logger):
        self.logger = logger
        self.price_limits = {
            'min_price': 1.0,
            'max_price': 10000.0,
            'min_volume': 1000
        }
        
    def validate_stock_data(self, df: pd.DataFrame, stock_id: str) -> bool:
        """驗證股票資料完整性
        
        Args:
            df: 股票資料DataFrame
            stock_id: 股票代碼
            
        Returns:
            bool: 驗證是否通過
        """
        try:
            # 股票代碼格式驗證
            if not self._validate_stock_id(stock_id):
                self.logger.error(f"股票代碼 {stock_id} 格式無效")
                return False
            
            # 日期連續性檢查
            if not self._validate_date_continuity(df):
                self.logger.error(f"股票 {stock_id} 日期不連續")
                return False
            
            # 價格合理性驗證
            if not self._validate_price_data(df):
                self.logger.error(f"股票 {stock_id} 價格資料異常")
                return False
            
            # 成交量檢查
            if not self._validate_volume_data(df):
                self.logger.error(f"股票 {stock_id} 成交量資料異常")
                return False
            
            return True
            
        except Exception as e:
            self.logger.error(f"驗證股票 {stock_id} 資料時發生錯誤: {str(e)}")
            return False
    
    def _validate_stock_id(self, stock_id: str) -> bool:
        """驗證股票代碼格式"""
        try:
            stock_id = str(stock_id).strip()
            if not stock_id.isdigit():
                return False
            if len(stock_id) < 4 or len(stock_id) > 6:
                return False
            return True
        except Exception:
            return False
    
    def _validate_date_continuity(self, df: pd.DataFrame) -> bool:
        """檢查日期連續性"""
        try:
            df['日期'] = pd.to_datetime(df['日期'])
            date_diffs = df['日期'].sort_values().diff().dt.days
            
            # 排除假日後的間隔
            irregular_gaps = date_diffs[(date_diffs > 5) & (date_diffs < 60)]
            
            if len(irregular_gaps) > 0:
                self.logger.warning(f"發現 {len(irregular_gaps)} 處異常的日期間隔")
                return False
                
            return True
            
        except Exception as e:
            self.logger.error(f"檢查日期連續性時發生錯誤: {str(e)}")
            return False
    
    def _validate_price_data(self, df: pd.DataFrame) -> bool:
        """驗證價格資料"""
        try:
            price_cols = ['開盤價', '最高價', '最低價', '收盤價']
            
            for col in price_cols:
                if col not in df.columns:
                    return False
                    
                price_data = df[col]
                
                # 檢查價格範圍
                if (price_data < self.price_limits['min_price']).any() or \
                   (price_data > self.price_limits['max_price']).any():
                    return False
                    
                # 檢查價格邏輯關係
                if not all(df['最高價'] >= df['收盤價']) or \
                   not all(df['收盤價'] >= df['最低價']):
                    return False
                    
            return True
            
        except Exception:
            return False
    
    def _validate_volume_data(self, df: pd.DataFrame) -> bool:
        """驗證成交量資料"""
        try:
            if '成交股數' not in df.columns:
                return False
                
            volume_data = df['成交股數']
            
            # 檢查基本有效性
            if (volume_data < 0).any():
                return False
                
            # 檢查是否有異常的成交量
            mean_volume = volume_data.mean()
            std_volume = volume_data.std()
            abnormal_volume = volume_data[volume_data > mean_volume + 5 * std_volume]
            
            if len(abnormal_volume) > 0:
                self.logger.warning(f"發現 {len(abnormal_volume)} 筆異常成交量")
                
            return True
            
        except Exception:
            return False
    
    @staticmethod
    def _calculate_distribution_metrics(series: pd.Series) -> Dict[str, float]:
        """計算分布指標"""
        return {
            'mean': series.mean(),
            'std': series.std(),
            'skew': series.skew(),
            'kurtosis': series.kurtosis(),
            'unique_ratio': len(series.unique()) / len(series)
        }
    
    @staticmethod
    def _calculate_validity_metrics(series: pd.Series) -> Dict[str, float]:
        """計算有效性指標"""
        Q1 = series.quantile(0.25)
        Q3 = series.quantile(0.75)
        IQR = Q3 - Q1
        
        return {
            'outlier_ratio': ((series < (Q1 - 1.5 * IQR)) | 
                            (series > (Q3 + 1.5 * IQR))).mean(),
            'value_range': series.max() - series.min(),
            'zero_ratio': (series == 0).mean()
        }

In [29]:
class TestFeatureGenerator:
    """測試用特徵生成器類別
    
    主要功能：
    1. 生成技術分析特徵
    2. 處理產業資料
    3. 數據驗證和清理
    """
    
    def __init__(self, config: TestFeatureConfig):
        """初始化特徵生成器
        
        Args:
            config: TestFeatureConfig 配置類別實例
        """
        self.config = config
        self.logger = logging.getLogger(__name__)
        # 產業資料快取
        self._industry_mapping_cache = None
        self._valid_industries = None

    @staticmethod
    def _verify_stock_id(stock_id: str) -> bool:
        """驗證股票代碼格式
        
        Args:
            stock_id (str): 股票代碼
            
        Returns:
            bool: 驗證是否通過
        """
        try:
            # 去除空白
            stock_id = str(stock_id).strip()
            
            # 基本格式檢查
            if not stock_id.isdigit():
                return False
                
            # 長度檢查 (台股代碼通常為4-6碼)
            if len(stock_id) < 4 or len(stock_id) > 6:
                return False
                
            return True
            
        except Exception:
            return False
    
    def _process_technical_features(self, df: pd.DataFrame, stock_id: str) -> pd.DataFrame:
        """處理技術指標相關特徵"""
        try:
            # 合併技術指標
            df = self._merge_technical_indicators(df, stock_id)
            
            # 計算技術特徵
            if 'slowk' in df.columns and 'slowd' in df.columns:
                df['KD_差值'] = df['slowk'] - df['slowd']
            
            # 計算均線糾結度
            df['均線糾結度'] = self._calculate_ma_convergence(df)
            
            # 計算動能指標
            df = self._calculate_momentum_features(df)
            
            # 計算技術綜合評分
            df['技術綜合評分'] = self._calculate_technical_score(df)
            
            return df
            
        except Exception as e:
            self.logger.error(f"處理技術特徵時發生錯誤: {str(e)}")
            return df

    

    def validate_data_structure(self) -> bool:
        """驗證數據結構的完整性"""
        try:
            # 1. 檢查必要的目錄結構
            required_dirs = [
                self.config.meta_data_path,
                self.config.test_data_path,
                self.config.features_path,
                self.config.industry_analysis_path
            ]
            
            for dir_path in required_dirs:
                if not dir_path.exists():
                    self.logger.error(f"找不到必要目錄: {dir_path}")
                    return False
                    
            # 2. 驗證檔案存在
            required_files = [
                self.config.meta_data_path / 'market_index.csv',
                self.config.meta_data_path / 'industry_index.csv',
                self.config.meta_data_path / 'companies.csv'
            ]
            
            for file_path in required_files:
                if not file_path.exists():
                    self.logger.error(f"找不到必要檔案: {file_path}")
                    return False
                    
            # 3. 確認檔案可讀取
            for file_path in required_files:
                try:
                    pd.read_csv(file_path, nrows=5)
                except Exception as e:
                    self.logger.error(f"無法讀取檔案 {file_path}: {str(e)}")
                    return False
                    
            # 4. 驗證資料內容的正確性
            companies_df = pd.read_csv(self.config.meta_data_path / 'companies.csv')
            
            # 檢查必要欄位
            required_columns = ['industry_category', 'stock_id', 'stock_name', 'type']
            if not all(col in companies_df.columns for col in required_columns):
                self.logger.error(f"companies.csv 缺少必要欄位: {required_columns}")
                return False
                
            # 過濾非ETF資料
            non_etf_df = companies_df[companies_df['type'].str.lower() != 'etf']
            
            # 只輸出關鍵統計信息
            self.logger.info(f"資料統計:")
            self.logger.info(f"- 可用股票數: {len(non_etf_df)}")
            self.logger.info(f"- 產業類別數: {len(non_etf_df['industry_category'].unique())}")
            
            # 檢查日期範圍
            industry_df = pd.read_csv(self.config.meta_data_path / 'industry_index.csv')
            industry_df['日期'] = pd.to_datetime(industry_df['日期'])
            self.logger.info(f"數據日期範圍: {industry_df['日期'].min():%Y-%m-%d} 到 {industry_df['日期'].max():%Y-%m-%d}")
            
            return True
            
        except Exception as e:
            self.logger.error(f"驗證數據結構時發生錯誤: {str(e)}")
            return False
    
    def validate_data_completeness(self, df: pd.DataFrame, stock_id: str) -> bool:
        """驗證數據的完整性"""
        try:
            # 1. 檢查時間序列的連續性
            df['日期'] = pd.to_datetime(df['日期'])
            date_gaps = df['日期'].sort_values().diff().dt.days.dropna()
            
            # 允許的最大間隔(扣除假日)
            MAX_GAP = 5
            abnormal_gaps = date_gaps[date_gaps > MAX_GAP].count()
            if abnormal_gaps > 0:
                self.logger.warning(f"股票 {stock_id} 有 {abnormal_gaps} 個異常的時間間隔")
                
            # 2. 驗證價格數據的有效性
            price_cols = ['開盤價', '最高價', '最低價', '收盤價']
            for col in price_cols:
                # 檢查空值比例
                null_ratio = df[col].isnull().mean()
                if null_ratio > 0.1:  # 空值比例不超過10%
                    self.logger.warning(f"股票 {stock_id} 的 {col} 有 {null_ratio:.1%} 的空值")
                    
                # 檢查價格為0或負數的情況
                invalid_prices = (df[col] <= 0).sum()
                if invalid_prices > 0:
                    self.logger.warning(f"股票 {stock_id} 的 {col} 有 {invalid_prices} 筆無效價格")
                    
            # 3. 確認成交量資料的合理性
            volume_cols = ['成交股數', '成交筆數', '成交金額']
            for col in volume_cols:
                # 檢查空值
                if df[col].isnull().any():
                    self.logger.warning(f"股票 {stock_id} 的 {col} 包含空值")
                    
                # 檢查負值
                if (df[col] < 0).any():
                    self.logger.warning(f"股票 {stock_id} 的 {col} 包含負值")
                    
            # 4. 基本合理性檢查
            if not all(df['最高價'] >= df['收盤價']) or not all(df['收盤價'] >= df['最低價']):
                self.logger.warning(f"股票 {stock_id} 的價格關係異常")
                
            return True
            
        except Exception as e:
            self.logger.error(f"驗證數據完整性時發生錯誤: {str(e)}")
            return False
    
    def _safe_convert_numeric(self, x) -> float:
        """安全的數值轉換方法"""
        try:
            # 1. 處理空值
            if pd.isna(x) or str(x).strip() in ['', '--', 'nan', 'NULL']:
                return np.nan
                
            # 2. 處理字串值
            if isinstance(x, str):
                # 移除千分位符號
                x = x.replace(',', '')
                # 處理百分比
                if '%' in x:
                    return float(x.replace('%', '')) / 100
                    
            # 3. 轉換數值
            value = float(x)
            
            # 4. 驗證合理性
            if value > 1e10 or value < -1e10:  # 超出合理範圍
                return np.nan
                
            return value
            
        except (ValueError, TypeError):
            return np.nan

    def generate_features(self, df: pd.DataFrame, stock_id: str) -> pd.DataFrame:
        """生成特徵並與現有技術指標合併
        
        Args:
            df (pd.DataFrame): 原始股票數據
            stock_id (str): 股票代碼
            
        Returns:
            pd.DataFrame: 包含所有特徵的數據框
        """
        try:
            # 1. 數據預處理
            df = self._preprocess_data(df)
            if df is None:
                self.logger.error(f"股票 {stock_id} 預處理失敗")
                return None

            # 2. 數據驗證
            if not self.validate_data_completeness(df, stock_id):
                self.logger.error(f"股票 {stock_id} 數據驗證失敗")
                return None

            # 3. 生成基礎特徵
            df = self._add_volume_features(df)
            df = self._add_volatility_features(df)
            df = self._add_trend_features(df)
            
            # 4. 讀取和合併技術指標
            df = self._merge_technical_indicators(df, stock_id)
            df = self._add_technical_features(df)
            
            # 5. 處理產業資料
            industry_df = self._process_industry_index_data()
            if industry_df is not None:
                df = self._add_industry_features(df, industry_df, stock_id)
            else:
                self.logger.warning(f"股票 {stock_id} 無法處理產業特徵")
                
            # 6. 特徵後處理
            df = self._post_process_features(df)
            
            # 7. 清理記憶體
            self._check_memory_usage()
            
            return df
            
        except Exception as e:
            self.logger.error(f"生成特徵時發生錯誤: {str(e)}")
            return None

    def _preprocess_data(self, df: pd.DataFrame) -> pd.DataFrame:
        """數據預處理
        
        Args:
            df (pd.DataFrame): 原始數據框
            
        Returns:
            pd.DataFrame: 預處理後的數據框
        """
        try:
            df = df.copy(deep=False)
            # 優化類別欄位
            category_columns = ['證券代號', '證券名稱']
            for col in category_columns:
                if col in df.columns:
                    df[col] = df[col].astype('category')
                
            # 1. 確保必要欄位存在
            required_columns = [
                '證券代號', '日期', '開盤價', '最高價', '最低價', 
                '收盤價', '成交股數', '成交金額'
            ]
            
            missing_cols = [col for col in required_columns if col not in df.columns]
            if missing_cols:
                self.logger.error(f"缺少必要欄位: {missing_cols}")
                return None
            
            # 2. 處理日期格式
            df['日期'] = pd.to_datetime(df['日期'])
            
            # 3. 處理數值欄位
            price_columns = ['開盤價', '最高價', '最低價', '收盤價']
            volume_columns = ['成交股數', '成交筆數', '成交金額']
            
            for col in price_columns + volume_columns:
                if col in df.columns:
                    df[col] = df[col].apply(self._safe_convert_numeric)
            
            # 4. 處理成交金額空值
            if '成交金額' in df.columns:
                null_count = df['成交金額'].isnull().sum()
                if null_count > 0:
                    self.logger.info(f"處理成交金額空值(共 {null_count} 筆)...")
                    # 使用成交股數和收盤價計算
                    df['成交金額'] = df.apply(
                        lambda row: row['成交股數'] * row['收盤價'] 
                        if pd.isnull(row['成交金額']) 
                        else row['成交金額'],
                        axis=1
                    )
            
            # 5. 驗證數據基本合理性
            for col in price_columns:
                # 檢查價格為0或負數
                invalid_prices = (df[col] <= 0).sum()
                if invalid_prices > 0:
                    self.logger.warning(f"{col} 有 {invalid_prices} 筆資料小於等於0")
                
                # 檢查價格極端值
                mean = df[col].mean()
                std = df[col].std()
                extreme_prices = (df[col] > mean + 3 * std).sum()
                if extreme_prices > 0:
                    self.logger.warning(f"{col} 有 {extreme_prices} 筆極端值")
            
            # 6. 檢查價格邏輯關係
            price_errors = (
                (df['最高價'] < df['最低價']) |
                (df['開盤價'] < df['最低價']) |
                (df['開盤價'] > df['最高價']) |
                (df['收盤價'] < df['最低價']) |
                (df['收盤價'] > df['最高價'])
            ).sum()
            
            if price_errors > 0:
                self.logger.warning(f"發現 {price_errors} 筆價格邏輯錯誤")
                
            # 7. 處理成交量的合理性
            for col in volume_columns:
                if col in df.columns:
                    # 處理負值
                    negative_volumes = (df[col] < 0).sum()
                    if negative_volumes > 0:
                        self.logger.warning(f"{col} 有 {negative_volumes} 筆負值")
                        df[col] = df[col].abs()
                    
                    # 處理異常大的值
                    vol_mean = df[col].mean()
                    vol_std = df[col].std()
                    extreme_volumes = (df[col] > vol_mean + 5 * vol_std).sum()
                    if extreme_volumes > 0:
                        self.logger.warning(f"{col} 有 {extreme_volumes} 筆異常大的值")
            
            # 8. 檢查時間序列的連續性
            date_gaps = df['日期'].sort_values().diff().dt.days
            unusual_gaps = date_gaps[date_gaps > 5].count()  # 超過5天的間隔
            if unusual_gaps > 0:
                self.logger.warning(f"發現 {unusual_gaps} 處異常的時間間隔")
            
            # 9. 資料排序
            df = df.sort_values(['證券代號', '日期'])
            
            # 10. 檢查最終的空值情況
            null_stats = df[required_columns].isnull().sum()
            if null_stats.any():
                for col, null_count in null_stats[null_stats > 0].items():
                    self.logger.warning(f"{col} 仍有 {null_count} 筆空值")
            
            return df
            
        except Exception as e:
            self.logger.error(f"數據預處理失敗: {str(e)}")
            self.logger.error(traceback.format_exc())  # 加入堆疊追蹤
            return None

    def _add_volume_features(self, df: pd.DataFrame) -> pd.DataFrame:
        """添加量能特徵
        
        計算特徵：
        - 量比
        - 量增率
        - 量能趨勢
        
        Args:
            df (pd.DataFrame): 數據框
            
        Returns:
            pd.DataFrame: 添加量能特徵後的數據框
        """
        try:
            params = self.config.FEATURE_PARAMS['volume']
            short_period = params['short_period']
            long_period = params['long_period']
            
            # 計算量比
            df['量比'] = df['成交股數'] / df['成交股數'].rolling(long_period).mean()
            
            # 計算量增率
            df['量增率'] = df['成交股數'].pct_change()
            
            # 計算量能趨勢
            short_ma = df['成交股數'].rolling(short_period).mean()
            long_ma = df['成交股數'].rolling(long_period).mean()
            df['量能趨勢'] = short_ma / long_ma
            
            return df
            
        except Exception as e:
            self.logger.error(f"計算量能特徵時發生錯誤: {str(e)}")
            return df

    def _add_volatility_features(self, df: pd.DataFrame) -> pd.DataFrame:
        """添加波動特徵
        
        計算特徵：
        - 日內波動率
        - 振幅
        - 漲跌幅
        - 波動率趨勢
        
        Args:
            df (pd.DataFrame): 數據框
            
        Returns:
            pd.DataFrame: 添加波動特徵後的數據框
        """
        try:
            params = self.config.FEATURE_PARAMS['volatility']
            short_period = params['short_period']
            long_period = params['long_period']
            
            # 計算日內波動率
            df['日內波動率'] = (df['最高價'] - df['最低價']) / df['開盤價']
            
            # 計算振幅
            df['振幅'] = (df['最高價'] - df['最低價']) / df['收盤價'].shift(1)
            
            # 計算漲跌幅
            df['漲跌幅'] = df['收盤價'].pct_change()
            
            # 計算波動率趨勢
            short_vol = df['收盤價'].rolling(short_period).std()
            long_vol = df['收盤價'].rolling(long_period).std()
            df['波動率趨勢'] = short_vol / long_vol
            
            return df
            
        except Exception as e:
            self.logger.error(f"計算波動特徵時發生錯誤: {str(e)}")
            return df

    def _add_trend_features(self, df: pd.DataFrame) -> pd.DataFrame:
        """添加趨勢特徵
        
        計算特徵：
        - 趨勢強度
        - 通道寬度變化
        - 趨勢動能
        - 趨勢持續性
        
        Args:
            df (pd.DataFrame): 數據框
            
        Returns:
            pd.DataFrame: 添加趨勢特徵後的數據框
        """
        try:
            params = self.config.FEATURE_PARAMS['trend']
            ma_period = params['ma_period']
            
            # 計算趨勢強度
            ma = df['收盤價'].rolling(ma_period).mean()
            df['趨勢強度'] = (df['收盤價'] - ma) / ma
            
            # 計算布林通道寬度變化
            std = df['收盤價'].rolling(window=20).std()
            upper = ma + (2 * std)
            lower = ma - (2 * std)
            
            df['通道寬度'] = (upper - lower) / ma
            df['通道寬度變化'] = df['通道寬度'].pct_change()
            
            # 計算趨勢動能
            df['趨勢動能'] = df['收盤價'].diff(ma_period) / df['收盤價'].shift(ma_period)
            
            # 計算趨勢持續性
            df['趨勢持續性'] = df['趨勢強度'].rolling(ma_period).sum()
            
            return df
            
        except Exception as e:
            self.logger.error(f"計算趨勢特徵時發生錯誤: {str(e)}")
            return df

    def _merge_technical_indicators(self, df: pd.DataFrame, stock_id: str) -> pd.DataFrame:
        """合併技術指標數據
        
        Args:
            df (pd.DataFrame): 數據框
            stock_id (str): 股票代碼
            
        Returns:
            pd.DataFrame: 合併技術指標後的數據框
        """
        try:
            tech_file = Path(f"{self.config.BASE_DIR}/technical_analysis/{stock_id}_indicators.csv")
            
            if not tech_file.exists():
                self.logger.warning(f"找不到技術指標檔案: {tech_file}")
                tech_cols = ['SMA30', 'DEMA30', 'EMA30', 'RSI', 'MACD', 'MACD_signal',
                            'MACD_hist', 'slowk', 'slowd', 'TSF', 'middleband', 'SAR']
                for col in tech_cols:
                    df[col] = np.nan
                return df
            
            if tech_file.exists():
                tech_df = pd.read_csv(tech_file, dtype={'證券代號': str})
                
                tech_df['日期'] = pd.to_datetime(tech_df['日期'])
                df['日期'] = pd.to_datetime(df['日期'])
                
                tech_cols = ['SMA30', 'DEMA30', 'EMA30', 'RSI', 'MACD', 'MACD_signal',
                            'MACD_hist', 'slowk', 'slowd', 'TSF', 'middleband', 'SAR']
                
                df = pd.merge(
                    df,
                    tech_df[['證券代號', '日期'] + tech_cols],
                    on=['證券代號', '日期'],
                    how='left'
                )
                
                # 只輸出一次整體統計
                valid_ratios = {col: df[col].notna().mean() * 100 for col in tech_cols}
                avg_ratio = sum(valid_ratios.values()) / len(valid_ratios)
                self.logger.info(f"技術指標平均有效率: {avg_ratio:.1f}%")
                
            return df
            
        except Exception as e:
            self.logger.error(f"合併技術指標時發生錯誤: {str(e)}")
            return df

    def _add_technical_features(self, df: pd.DataFrame) -> pd.DataFrame:
        """添加技術面特徵
        
        計算特徵：
        - KD差值
        - 均線糾結度
        - RSI動能
        - MACD動能
        - 技術綜合評分
        
        Args:
            df (pd.DataFrame): 數據框
            
        Returns:
            pd.DataFrame: 添加技術特徵後的數據框
        """
        try:
            # 1. 計算KD差值
            if 'slowk' in df.columns and 'slowd' in df.columns:
                df['KD_差值'] = np.where(
                    df['slowk'].notna() & df['slowd'].notna(),
                    df['slowk'] - df['slowd'],
                    np.nan
                )
            
            # 2. 計算均線糾結度
            ma_cols = ['SMA30', 'DEMA30', 'EMA30']
            if all(col in df.columns for col in ma_cols):
                try:
                    deviations = []
                    base_ma = df['SMA30']
                    
                    # 檢查基準均線是否有效
                    if base_ma.notna().any():
                        for i in range(len(ma_cols)):
                            for j in range(i+1, len(ma_cols)):
                                # 安全的除法運算
                                deviation = np.where(
                                    df[ma_cols[i]].notna() & 
                                    df[ma_cols[j]].notna() & 
                                    (df[ma_cols[i]] != 0),
                                    abs(df[ma_cols[i]] - df[ma_cols[j]]) / df[ma_cols[i]],
                                    np.nan
                                )
                                deviations.append(pd.Series(deviation))
                        
                        # 計算糾結度
                        if deviations:
                            df['均線糾結度'] = 1 - pd.concat(deviations, axis=1).mean(axis=1)
                            # 限制範圍在 0-1 之間
                            df['均線糾結度'] = df['均線糾結度'].clip(0, 1)
                except Exception as e:
                    self.logger.warning(f"計算均線糾結度時發生錯誤: {str(e)}")
            
            # 3. 計算RSI動能
            if 'RSI' in df.columns:
                df['RSI_動能'] = self._safe_calculate_momentum(df['RSI'])
            
            # 4. 計算MACD動能
            if 'MACD_hist' in df.columns:
                df['MACD_動能'] = self._safe_calculate_momentum(df['MACD_hist'])
            
            # 5. 計算技術綜合評分
            tech_scores = []
            
            # RSI評分 (理想值接近50)
            if 'RSI' in df.columns:
                rsi_score = np.where(
                    df['RSI'].notna(),
                    1 - abs(df['RSI'] - 50) / 50,
                    np.nan
                )
                tech_scores.append(pd.Series(rsi_score))
            
            # MACD評分
            if 'MACD' in df.columns and 'MACD_signal' in df.columns:
                macd_diff = df['MACD'] - df['MACD_signal']
                macd_score = 1 / (1 + np.exp(-macd_diff))  # Sigmoid函數
                tech_scores.append(pd.Series(macd_score))
            
            # 均線糾結度評分
            if '均線糾結度' in df.columns:
                tech_scores.append(df['均線糾結度'])
            
            # 合併技術評分
            if tech_scores:
                df['技術綜合評分'] = pd.concat(tech_scores, axis=1).mean(axis=1)
                # 限制範圍在 0-1 之間
                df['技術綜合評分'] = df['技術綜合評分'].clip(0, 1)
            
            return df
            
        except Exception as e:
            self.logger.error(f"計算技術特徵時發生錯誤: {str(e)}")
            return df
    
    def _safe_calculate_momentum(self, series: pd.Series) -> pd.Series:
        """安全計算動能
        
        Args:
            series: 輸入數據序列
            
        Returns:
            pd.Series: 動能序列
        """
        try:
            # 計算變化率
            diff = series.diff()
            prev = series.shift(1)
            
            # 安全除法
            return np.where(
                prev.notna() & (prev != 0),
                diff / prev,
                np.nan
            )
        except Exception:
            return pd.Series(index=series.index)

    def _process_industry_index_data(self) -> pd.DataFrame:
        """處理產業指數數據
        
        Returns:
            pd.DataFrame: 處理後的產業指數數據，失敗則返回 None
        """
        try:
            # 讀取產業指數資料
            industry_file = self.config.meta_data_path / 'industry_index.csv'
            if not industry_file.exists():
                self.logger.error("找不到產業指數檔案")
                return None
                
            df = pd.read_csv(industry_file)
            
            # 處理日期格式
            df['日期'] = pd.to_datetime(df['日期'])
            
            # 處理漲跌欄位的空值
            # 1. 若有漲跌點數，根據正負來填補漲跌
            df.loc[df['漲跌點數'] > 0, '漲跌'] = '+'
            df.loc[df['漲跌點數'] < 0, '漲跌'] = '-'
            df.loc[df['漲跌點數'] == 0, '漲跌'] = ' '
            
            # 2. 仍有空值的話，使用前值填充
            df['漲跌'] = df['漲跌'].fillna(method='ffill')
            
            # 驗證數據
            null_columns = df.columns[df.isnull().any()].tolist()
            if null_columns:
                self.logger.warning(f"這些欄位仍含有空值: {null_columns}")
                
            return df
            
        except Exception as e:
            self.logger.error(f"處理產業指數數據時發生錯誤: {str(e)}")
            return None

    def _get_industry_name(self, stock_id: str) -> List[str]:
        """獲取股票所屬產業
        
        Args:
            stock_id (str): 股票代碼
            
        Returns:
            List[str]: 標準化後的產業分類列表
        """
        try:
            # 初始化或更新產業映射快取
            if not hasattr(self, '_industry_mapping_cache') or self._industry_mapping_cache is None:
                self._industry_mapping_cache = {}
                
                company_file = self.config.meta_data_path / 'companies.csv'
                
                if not company_file.exists():
                    self.logger.error(f"找不到公司產業分類檔案: {company_file}")
                    return []
                
                # 讀取公司資料
                company_data = pd.read_csv(
                    company_file,
                    dtype={
                        'industry_category': str,
                        'stock_id': str,
                        'stock_name': str
                    }
                )
                
                # 簡化的映射建立
                for _, row in company_data.iterrows():
                    if pd.notna(row['industry_category']):
                        self._industry_mapping_cache[row['stock_id']] = [row['industry_category']]
            
            # 從快取中獲取產業資訊
            stock_id = str(stock_id).strip()
            industries = self._industry_mapping_cache.get(stock_id, [])
            
            if industries:
                # 顯示產業對應資訊
                industry_str = ', '.join(industries)
                self.logger.info(f"股票 {stock_id} 對應的產業: {industry_str}")
                self.logger.info(f"股票 {stock_id} 對應 {len(industries)} 個產業指數")
                return industries
                
            self.logger.warning(f"股票 {stock_id} 未找到產業分類")
            return []
            
        except Exception as e:
            self.logger.error(f"獲取產業分類時發生錯誤: {str(e)}")
            return []
    
    def _standardize_industry_name(self, name: str) -> str:
        """標準化產業名稱
        
        Args:
            name (str): 原始產業名稱
            
        Returns:
            str: 標準化後的產業名稱
        """
        if not name:
            return ""
            
        # 1. 移除指數相關後綴
        name = name.replace('類報酬指數', '')\
                   .replace('類指數', '')\
                   .replace('類日報酬兩倍指數', '')\
                   .replace('類日報酬反向一倍指數', '')\
                   .replace('類兩倍槓桿指數', '')\
                   .replace('類反向指數', '')
        
        # 2. 移除產業相關後綴
        name = name.replace('工業', '')\
                   .replace('業', '')\
                   .replace('事業', '')
        
        # 3. 移除特殊字詞
        name = name.replace('類', '')
        
        # 4. 最後進行清理
        return name.strip()

    def _add_industry_features(self, df: pd.DataFrame, industry_df: pd.DataFrame, stock_id: str) -> pd.DataFrame:
        """添加產業特徵
        
        Args:
            df (pd.DataFrame): 原始數據框
            industry_df (pd.DataFrame): 產業指數數據框
            stock_id (str): 股票代碼
            
        Returns:
            pd.DataFrame: 添加產業特徵後的數據框
        """
        try:
            # 1. 取得產業分類
            industries = self._get_industry_name(stock_id)
            if not industries:
                self.logger.warning(f"股票 {stock_id} 無法取得產業分類")
                return df
            
            df = df.copy()
            df['日期'] = pd.to_datetime(df['日期'])
            industry_df['日期'] = pd.to_datetime(industry_df['日期'])
            
            # 處理產業特徵(保持原有邏輯)
            for i, industry in enumerate(industries):
                suffix = f"_{i+1}" if len(industries) > 1 else ""
                industry_data = industry_df[industry_df['指數名稱'].str.contains(industry, na=False)].copy()
                
                if industry_data.empty:
                    continue
                    
                if industry_data.duplicated('日期').any():
                    industry_data = industry_data.groupby('日期').agg({
                        '收盤指數': 'mean',
                        '漲跌百分比': 'mean'
                    }).reset_index()
                
                industry_data.set_index('日期', inplace=True)
                df_dates = pd.Index(df['日期'].unique())
                industry_data = industry_data.reindex(df_dates)
                
                # 添加特徵
                df[f'產業_報酬率{suffix}'] = industry_data['漲跌百分比'].values
                price_series = industry_data['收盤指數']
                df[f'產業_波動率{suffix}'] = price_series.pct_change().rolling(window=20).std().values
                df[f'產業_強度排名{suffix}'] = price_series.rank(pct=True).values
                df[f'產業_動能得分{suffix}'] = price_series.pct_change(periods=5).rolling(window=20).mean().values
                
            # 只在最後輸出一次產業特徵統計
            industry_features = [col for col in df.columns if '產業_' in col]
            if industry_features:
                self.logger.info(f"已新增 {len(industry_features)} 個產業特徵")
                
            return df
            
        except Exception as e:
            self.logger.error(f"添加產業特徵時發生錯誤: {str(e)}")
            return df

    def _post_process_features(self, df: pd.DataFrame) -> pd.DataFrame:
        """特徵後處理
        
        處理內容包括：
        - 處理無限值
        - 填補重要特徵的空值
        - 特徵異常值處理
        
        Args:
            df (pd.DataFrame): 原始數據框
            
        Returns:
            pd.DataFrame: 後處理完成的數據框
        """
        try:
            # 1. 處理無限值
            df = df.replace([np.inf, -np.inf], np.nan)
            
            # 2. 處理重要特徵的空值
            important_features = [
                '技術綜合評分', '產業相對強度', '漲跌幅', '量增率', 
                'RSI_動能', 'MACD_動能', '波動率趨勢'
            ]
            
            # 確保特徵存在再進行填充
            existing_features = [f for f in important_features if f in df.columns]
            if existing_features:
                df[existing_features] = df[existing_features].fillna(method='ffill')
            
            # 3. 處理異常值
            numeric_features = df.select_dtypes(include=[np.number]).columns
            for col in numeric_features:
                # 計算上下界
                Q1 = df[col].quantile(0.25)
                Q3 = df[col].quantile(0.75)
                IQR = Q3 - Q1
                lower_bound = Q1 - 1.5 * IQR
                upper_bound = Q3 + 1.5 * IQR
                
                # 使用界限值替換異常值
                df[col] = df[col].clip(lower_bound, upper_bound)
                
            return df
            
        except Exception as e:
            self.logger.error(f"特徵後處理時發生錯誤: {str(e)}")
            return df

    def save_features(self, df: pd.DataFrame, stock_id: str) -> bool:
        """儲存生成的特徵
        
        Args:
            df (pd.DataFrame): 特徵資料
            stock_id (str): 股票代碼
            
        Returns:
            bool: 儲存是否成功
        """
        try:
            # 確保目錄存在
            save_dir = self.config.features_path
            save_dir.mkdir(parents=True, exist_ok=True)
            
            # 從 companies.csv 取得標準產業名稱
            company_file = self.config.meta_data_path / 'companies.csv'
            if company_file.exists():
                company_data = pd.read_csv(company_file, dtype={'stock_id': str})
                industry_info = company_data[company_data['stock_id'] == stock_id]
                industry_name = (industry_info['industry_category'].iloc[0] 
                               if not industry_info.empty else 'unknown')
            else:
                industry_name = 'unknown'
            
            # 取得日期範圍
            start_date = df['日期'].min().strftime('%Y%m%d')
            end_date = df['日期'].max().strftime('%Y%m%d')
            current_date = datetime.now().strftime('%Y%m%d')
            
            # 構建檔案名稱時使用標準產業名稱
            filename = f"{stock_id}_{industry_name}_{start_date}_{end_date}_{current_date}.csv"
            save_path = save_dir / filename
            
            # 儲存特徵
            df.to_csv(save_path, index=False, encoding='utf-8-sig')
            self.logger.info(f"特徵已儲存至: {save_path}")
            
            return True
            
        except Exception as e:
            self.logger.error(f"儲存特徵時發生錯誤: {str(e)}")
            return False

    def save_combined_features(self, stock_ids: List[str]) -> bool:
        """合併並儲存所有股票的特徵
        
        Args:
            stock_ids (List[str]): 要合併的股票代碼清單
            
        Returns:
            bool: 是否成功儲存
        """
        try:
            all_features = []
            for stock_id in stock_ids:
                # 找出最新的特徵檔案
                feature_files = list(self.config.features_path.glob(f"{stock_id}_*.csv"))
                if feature_files:
                    latest_file = max(feature_files, key=lambda x: x.stat().st_mtime)
                    df = pd.read_csv(latest_file)
                    all_features.append(df)
                else:
                    self.logger.warning(f"找不到股票 {stock_id} 的特徵檔案")
            
            if not all_features:
                self.logger.error("沒有找到任何特徵檔案")
                return False
            
            # 合併所有特徵
            combined_df = pd.concat(all_features, ignore_index=True)
            
            # 先將日期欄位轉換為 datetime
            combined_df['日期'] = pd.to_datetime(combined_df['日期'])
            
            # 取得日期範圍
            start_date = combined_df['日期'].min().strftime('%Y%m%d')
            end_date = combined_df['日期'].max().strftime('%Y%m%d')
            current_date = datetime.now().strftime('%Y%m%d')
            
            # 將日期轉回字串格式以存檔
            combined_df['日期'] = combined_df['日期'].dt.strftime('%Y-%m-%d')
            
            # 構建輸出檔名
            output_filename = f"combined_features_{start_date}_{end_date}_{current_date}.csv"
            output_path = self.config.features_path / output_filename
            
            # 儲存合併後的特徵
            combined_df.to_csv(output_path, index=False, encoding='utf-8-sig')
            self.logger.info(f"已合併儲存特徵至: {output_path}")
            
            return True
            
        except Exception as e:
            self.logger.error(f"合併儲存特徵時發生錯誤: {str(e)}")
            return False

    def _check_memory_usage(self):
        """檢查記憶體使用情況"""
        try:
            # 取得目前程序的記憶體使用量(GB)
            current_memory = psutil.Process().memory_info().rss / (1024 * 1024 * 1024)
            
            WARNING_THRESHOLD = 6
            CRITICAL_THRESHOLD = 8
            
            if current_memory > CRITICAL_THRESHOLD:
                self.logger.error(f"記憶體使用量達到危險水平: {current_memory:.2f}GB")
                # 執行垃圾回收
                gc.collect()
                # 檢查回收效果
                new_memory = psutil.Process().memory_info().rss / (1024 * 1024 * 1024)
                if new_memory > current_memory * 0.9:  # 如果回收效果不明顯
                    self.logger.error("記憶體回收效果不顯著，建議中斷處理")
                    return False
                
            return current_memory
            
        except Exception as e:
            self.logger.error(f"檢查記憶體使用時發生錯誤: {str(e)}")
            return None

    def _clear_temp_data(self):
        """清理暫存資料"""
        try:
            # 清理產業資料快取
            self._industry_mapping_cache = None
            self._valid_industries = None
            
            # 觸發垃圾回收
            gc.collect()
            
        except Exception as e:
            self.logger.error(f"清理暫存資料時發生錯誤: {str(e)}")

In [30]:
class FeatureLogger:
    def __init__(self, config=None):
        self.logger = logging.getLogger('FeatureGenerator')
        self.logger.setLevel(logging.INFO)
        
        # 移除所有現有的 handlers
        self.logger.handlers = []
        
        # 添加控制台處理器
        console_handler = logging.StreamHandler()
        console_handler.setFormatter(logging.Formatter('%(message)s'))
        self.logger.addHandler(console_handler)
        
        # 如果有配置，添加檔案處理器
        if config:
            log_file = config.get_log_path()
            file_handler = logging.FileHandler(log_file, encoding='utf-8')
            file_handler.setFormatter(
                logging.Formatter('%(asctime)s - %(message)s')
            )
            self.logger.addHandler(file_handler)
    
    def info(self, msg): self.logger.info(msg)
    def warning(self, msg): self.logger.warning(msg)
    def error(self, msg): self.logger.error(msg)

In [31]:
def main():
    """主程序"""
    # 設定日誌層級
    logging.basicConfig(level=logging.INFO)
    # 過濾重複的 FeatureGenerator 日誌
    logging.getLogger('FeatureGenerator').setLevel(logging.WARNING)
    
    try:
        # 初始化
        config = TestFeatureConfig()
        logger = FeatureLogger(config)
        logger.info("開始執行特徵生成主程序")
        
        # 驗證配置和數據結構
        generator = TestFeatureGenerator(config)
        if not config.validate_config():
            logger.warning("配置驗證發現問題，但繼續執行")
            
        if not generator.validate_data_structure():
            logger.warning("資料結構驗證發現問題，但繼續執行")
        
        # 檢查產業資料完整性
        logger.info("檢查產業資料完整性...")
        if not generator.validate_data_structure():
            logger.warning("產業資料驗證發現問題，但繼續執行")
        
        # 讀取股票主檔
        logger.info("讀取股票主檔...")
        try:
            main_df = pd.read_csv(
                config.get_stock_data_path(),
                dtype={
                    '證券代號': str,
                    '證券名稱': str,
                    '日期': str
                }
            )
            
            # 基本數據處理
            price_cols = ['開盤價', '最高價', '最低價', '收盤價']
            volume_cols = ['成交股數', '成交筆數', '成交金額']
            
            for col in price_cols + volume_cols:
                if col in main_df.columns:
                    main_df[col] = main_df[col].apply(safe_convert_numeric)
            
            logger.info(f"載入 {len(main_df):,} 筆資料")
            
            # 日期範圍驗證
            if not validate_date_range(main_df):
                logger.warning("日期範圍驗證發現異常，但繼續執行")
                
        except Exception as e:
            logger.error(f"讀取股票主檔失敗: {str(e)}")
            return False
        
        # 取得起始和結束日期
        date_range = main_df['日期'].agg(['min', 'max'])
        start_date = pd.to_datetime(date_range['min']).strftime('%Y%m%d')
        end_date = pd.to_datetime(date_range['max']).strftime('%Y%m%d')
        current_date = datetime.now().strftime('%Y%m%d')
            
        # 分組處理每支股票
        unique_stocks = sorted(main_df['證券代號'].unique())
        total_stocks = len(unique_stocks)
        processed_stocks = 0
        
        # 使用tqdm進度條
        with tqdm(total=total_stocks, desc="特徵生成進度") as pbar:
            for stock_id in unique_stocks:
                try:
                    # 處理單一股票
                    stock_df = main_df[main_df['證券代號'] == stock_id].copy()
                    result_df = generator.generate_features(stock_df, stock_id)
                    
                    if result_df is not None and not result_df.empty:
                        # 根據產業分析的命名邏輯構建檔名
                        if generator.save_features(result_df, stock_id):
                            processed_stocks += 1
                    
                except Exception as e:
                    logger.warning(f"處理股票 {stock_id} 時發生錯誤: {str(e)}")
                    continue
                
                pbar.update(1)
                pbar.set_postfix({
                    '成功': f"{processed_stocks}/{total_stocks}",
                    '比例': f"{(processed_stocks/total_stocks)*100:.1f}%"
                })
        
        # 檢查處理結果
        if processed_stocks > 0:
            logger.info(f"特徵生成完成，成功處理 {processed_stocks}/{total_stocks} 支股票")
            
            # 合併所有特徵並儲存
            combined_filename = f"combined_features_{start_date}_{end_date}_{current_date}.csv"
            if generator.save_combined_features(config.TEST_SETTING['test_stocks']):
                if verify_features(config):
                    return True
                else:
                    logger.warning("特徵驗證發現問題，但繼續執行")
                    return True
        else:
            logger.error("沒有成功處理的數據")
            return False
            
    except Exception as e:
        logger.error(f"主程序執行錯誤: {str(e)}")
        logger.error(f"錯誤詳情:\n{traceback.format_exc()}")
        return False

def validate_feature_quality(df: pd.DataFrame) -> Dict[str, float]:
    """驗證特徵質量"""
    try:
        if df is None or df.empty:
            return {"error": "空的DataFrame"}
            
        quality_metrics = {}
        
        # 1. 計算特徵的完整性
        null_ratios = df.isnull().mean()
        quality_metrics['completeness'] = {
            col: 1 - ratio 
            for col, ratio in null_ratios.items()
        }
        
        # 2. 檢查特徵的分布
        numeric_cols = df.select_dtypes(include=[np.number]).columns
        distribution_metrics = {}
        
        for col in numeric_cols:
            if df[col].notna().any():
                distribution_metrics[col] = {
                    'mean': df[col].mean(),
                    'std': df[col].std(),
                    'skew': df[col].skew(),
                    'kurtosis': df[col].kurtosis(),
                    'unique_ratio': len(df[col].unique()) / len(df)
                }
                
        quality_metrics['distribution'] = distribution_metrics
        
        # 3. 評估特徵的有效性
        validity_metrics = {}
        for col in numeric_cols:
            if df[col].notna().any():
                # 計算異常值比例
                Q1 = df[col].quantile(0.25)
                Q3 = df[col].quantile(0.75)
                IQR = Q3 - Q1
                outlier_ratio = (
                    (df[col] < (Q1 - 1.5 * IQR)) | 
                    (df[col] > (Q3 + 1.5 * IQR))
                ).mean()
                
                # 計算值域範圍
                value_range = df[col].max() - df[col].min()
                
                validity_metrics[col] = {
                    'outlier_ratio': outlier_ratio,
                    'value_range': value_range,
                    'zero_ratio': (df[col] == 0).mean()
                }
                
        quality_metrics['validity'] = validity_metrics
        
        return quality_metrics
        
    except Exception as e:
        logging.error(f"驗證特徵質量時發生錯誤: {str(e)}")
        return {}
        
def initialize_logging(config: TestFeatureConfig):
    """初始化日誌系統"""
    try:
        # 建立日誌目錄
        log_path = config.log_path
        log_path.mkdir(parents=True, exist_ok=True)
        
        # 設定日誌檔案
        log_file = log_path / f'feature_generator_{datetime.now():%Y%m%d}.log'
        
        # 設定日誌格式
        log_format = '%(asctime)s - %(name)s - %(levelname)s - %(message)s'
        date_format = '%Y-%m-%d %H:%M:%S'
        
        # 配置根日誌器
        logging.basicConfig(
            level=logging.INFO,
            format=log_format,
            datefmt=date_format,
            handlers=[
                # 檔案處理器
                logging.FileHandler(
                    log_file,
                    encoding='utf-8'
                ),
                # 控制台處理器
                logging.StreamHandler()
            ]
        )
        
        # 設定其他模組的日誌級別
        logging.getLogger('pandas').setLevel(logging.WARNING)
        logging.getLogger('numpy').setLevel(logging.WARNING)
        
        logging.info('日誌系統初始化完成')
        return True
        
    except Exception as e:
        print(f"初始化日誌系統時發生錯誤: {str(e)}")
        return False

def backup_data(config: TestFeatureConfig):
    """備份原始數據"""
    try:
        # 取得當前時間戳
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        
        # 建立備份目錄
        backup_dir = config.meta_data_path / 'backup' / timestamp
        backup_dir.mkdir(parents=True, exist_ok=True)
        
        # 需要備份的檔案清單
        files_to_backup = [
            'market_index.csv',
            'industry_index.csv',
            'companies.csv',
            'test_stock_data.csv'
        ]
        
        # 執行備份
        for filename in files_to_backup:
            source_path = config.meta_data_path / filename
            if source_path.exists():
                # 複製檔案到備份目錄
                shutil.copy2(
                    source_path,
                    backup_dir / filename
                )
                logging.info(f"已備份檔案: {filename}")
            else:
                logging.warning(f"找不到檔案: {filename}")
                
        # 清理舊的備份
        cleanup_old_backups(config)
        
        return True
        
    except Exception as e:
        logging.error(f"備份數據時發生錯誤: {str(e)}")
        return False

def cleanup_old_backups(config: TestFeatureConfig):
    """清理舊的備份檔案"""
    try:
        backup_root = config.meta_data_path / 'backup'
        if not backup_root.exists():
            return
            
        # 設定保留天數
        KEEP_DAYS = 7
        cutoff_date = datetime.now() - timedelta(days=KEEP_DAYS)
        
        # 取得所有備份目錄
        backup_dirs = [d for d in backup_root.iterdir() if d.is_dir()]
        
        for backup_dir in backup_dirs:
            try:
                # 從目錄名稱解析時間
                dir_time = datetime.strptime(backup_dir.name, '%Y%m%d_%H%M%S')
                
                # 如果超過保留天數，則刪除
                if dir_time < cutoff_date:
                    shutil.rmtree(backup_dir)
                    logging.info(f"已刪除舊的備份目錄: {backup_dir.name}")
                    
            except ValueError:
                logging.warning(f"無法解析備份目錄名稱: {backup_dir.name}")
                continue
                
    except Exception as e:
        logging.error(f"清理舊備份時發生錯誤: {str(e)}")

def cleanup_resources():
    """清理資源"""
    try:
        # 垃圾回收
        gc.collect()
        
        # 安全關閉檔案處理器
        for handler in logging.getLogger().handlers[:]:
            if hasattr(handler, 'close') and handler.stream is not None:
                try:
                    handler.close()
                    logging.getLogger().removeHandler(handler)
                except Exception:
                    continue
        
        # 檢查記憶體
        process = psutil.Process()
        memory_info = process.memory_info()
        memory_usage_gb = memory_info.rss / (1024 * 1024 * 1024)
        logging.info(f"最終記憶體使用量: {memory_usage_gb:.2f}GB")
        
        return True
        
    except Exception as e:
        logging.error(f"清理資源時發生錯誤: {str(e)}")
        return False

def safe_convert_numeric(x):
    """安全的數值轉換函數"""
    if pd.isna(x) or str(x).strip() in ['', '--']:
        return np.nan
    try:
        return float(str(x).replace(',', ''))
    except (ValueError, TypeError):
        return np.nan

def validate_numeric_data(df: pd.DataFrame, columns: List[str], logger) -> bool:
    """驗證數值欄位的有效性"""
    for col in columns:
        if col in df.columns:
            valid_ratio = df[col].notna().mean()
            if valid_ratio < 0.5:
                logger.warning(f"欄位 {col} 的有效數據比例過低: {valid_ratio:.2%}")
                return False
    return True

def validate_date_range(df: pd.DataFrame) -> bool:
    """驗證日期範圍有效性"""
    try:
        logger = FeatureLogger()  # 取得logger實例
        logger.info("開始驗證日期範圍...")
        
        # 檢查是否有日期欄位
        if '日期' not in df.columns:
            logger.error("缺少日期欄位")
            return False
            
        # 轉換日期格式
        df['日期'] = pd.to_datetime(df['日期'])
        
        # 基本資訊記錄
        start_date = df['日期'].min()
        end_date = df['日期'].max()
        total_days = (end_date - start_date).days
        
        logger.info(f"資料日期範圍: {start_date:%Y-%m-%d} 到 {end_date:%Y-%m-%d}")
        logger.info(f"總天數: {total_days}天")
        
        # 檢查每支股票的日期連續性
        stock_issues = []
        for stock_id in df['證券代號'].unique():
            stock_df = df[df['證券代號'] == stock_id]
            date_gaps = stock_df['日期'].sort_values().diff().dt.days.dropna()
            
            # 檢查工作日間隔
            abnormal_gaps = date_gaps[date_gaps > 5].count()  # 超過5天的間隔視為異常
            if abnormal_gaps > 0:
                stock_issues.append({
                    'stock_id': stock_id,
                    'abnormal_gaps': abnormal_gaps
                })
        
        # 如果有異常，記錄但不中斷程式
        if stock_issues:
            logger.warning("發現日期間隔異常:")
            for issue in stock_issues:
                logger.warning(f"股票 {issue['stock_id']} 有 {issue['abnormal_gaps']} 個異常間隔")
                
        return True
        
    except Exception as e:
        logger.error(f"驗證日期範圍時發生錯誤: {str(e)}")
        return False

def validate_feature_result(df: pd.DataFrame, stock_id: str) -> bool:
    """驗證特徵生成結果
    
    Args:
        df: 特徵結果DataFrame
        stock_id: 股票代碼
        
    Returns:
        bool: 驗證是否通過
    """
    try:
        # 1. 檢查基本欄位
        required_columns = [
            '證券代號', '日期', '開盤價', '最高價', '最低價', 
            '收盤價', '成交股數', '成交金額'
        ]
        if not all(col in df.columns for col in required_columns):
            return False
            
        # 2. 檢查數值欄位
        numeric_columns = [
            col for col in df.columns 
            if col not in ['證券代號', '證券名稱', '日期']
        ]
        
        for col in numeric_columns:
            # 檢查是否全為空值
            if df[col].isna().all():
                return False
                
            # 檢查是否包含無限值
            if np.isinf(df[col].replace([np.inf, -np.inf], np.nan)).any():
                return False
        
        # 3. 檢查時間連續性
        dates = pd.to_datetime(df['日期'])
        date_diff = dates.diff().dropna()
        if len(date_diff.unique()) > 3:  # 允許最多3種不同的時間間隔
            return False
            
        return True
        
    except Exception:
        return False

def generate_report(df: pd.DataFrame, total_stocks: int, processed_stocks: int, config: TestFeatureConfig):
    """生成處理報告"""
    logger = FeatureLogger(config)
    
    try:
        report_path = config.meta_data_path / 'feature_generation_report.txt'
        
        with open(report_path, 'w', encoding='utf-8') as f:
            # 基本資訊
            f.write("特徵生成處理報告\n")
            f.write(f"生成時間: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n")
            
            # 處理統計
            f.write(f"總股票數: {total_stocks}\n")
            f.write(f"成功處理: {processed_stocks}\n")
            f.write(f"處理比率: {(processed_stocks/total_stocks)*100:.2f}%\n\n")
            
            # 特徵統計
            f.write(f"特徵數量: {len(df.columns)}\n")
            f.write("\n特徵列表:\n")
            
            # 分類特徵列表
            feature_categories = {
                "基本資料": ['證券代號', '證券名稱', '日期', '開盤價', '最高價', '最低價', '收盤價', '成交股數'],
                "量能特徵": ['量比', '量增率', '量能趨勢', '量能波動', '量價背離'],
                "波動特徵": ['日內波動率', '振幅', '漲跌幅', '波動率趨勢'],
                "趨勢特徵": ['趨勢強度', '通道寬度變化', '趨勢動能', '趨勢持續性'],
                "技術特徵": ['RSI', 'MACD', 'KD_差值', '均線糾結度', 'RSI_動能', 'MACD_動能', '技術綜合評分'],
                "產業基本特徵": [col for col in df.columns if '產業_' in col],
                "產業風險特徵": [
                    col for col in df.columns 
                    if any(x in col for x in ['VaR', '波動率', '回撤', 'risk'])
                ],
                "產業動能特徵": [
                    col for col in df.columns 
                    if any(x in col for x in ['動能', '趨勢', '強度'])
                ],
                "相對表現特徵": [
                    col for col in df.columns 
                    if any(x in col for x in ['相對', '比較', '調整'])
                ]
            }
            
            for category, features in feature_categories.items():
                f.write(f"\n{category}:\n")
                present_features = [feat for feat in features if feat in df.columns]
                if present_features:
                    for feature in present_features:
                        # 計算非空值比例
                        non_null_ratio = (1 - df[feature].isnull().mean()) * 100
                        f.write(f"  - {feature} (有效數據: {non_null_ratio:.1f}%)\n")
                else:
                    f.write("  (無特徵)\n")
            
            # 產業覆蓋率分析
            f.write("\n產業覆蓋分析:\n")
            try:
                industry_columns = [col for col in df.columns if '產業_' in col]
                if industry_columns:
                    industries_covered = len(set([col.split('_')[1] for col in industry_columns]))
                    f.write(f"- 覆蓋產業數: {industries_covered}\n")
                    f.write("- 產業特徵示例:\n")
                    for ind in sorted(list(set([col.split('_')[1] for col in industry_columns])))[:5]:
                        f.write(f"  * {ind}\n")
                else:
                    f.write("- 未生成產業特徵\n")
            except Exception as e:
                f.write(f"- 產業分析錯誤: {str(e)}\n")
            
            # 數據範圍資訊
            f.write(f"\n數據時間範圍: {df['日期'].min()} 到 {df['日期'].max()}\n")
            
            # 特徵品質統計
            f.write("\n特徵品質統計:\n")
            null_stats = df.isnull().mean() * 100
            f.write(f"- 平均缺失率: {null_stats.mean():.2f}%\n")
            if null_stats.max() > 50:
                f.write("- 缺失率超過50%的特徵:\n")
                high_null_features = null_stats[null_stats > 50].index.tolist()
                for feat in high_null_features:
                    f.write(f"  * {feat}: {null_stats[feat]:.1f}%\n")
            
        logger.info(f"處理報告已保存至: {report_path}")
        
    except Exception as e:
        logger.error(f"生成報告時發生錯誤: {str(e)}")

def verify_features(config: TestFeatureConfig):
    """驗證特徵生成結果"""
    try:
        logger = FeatureLogger(config)
        
        # 使用最新生成的合併檔案
        current_date = datetime.now().strftime('%Y%m%d')
        combined_files = list(config.features_path.glob(f"combined_features_*.csv"))
        
        if not combined_files:
            logger.error("找不到合併特徵檔案")
            return False
            
        # 使用最新的合併檔案
        latest_file = max(combined_files, key=lambda x: x.stat().st_mtime)
        df = pd.read_csv(latest_file)
        
        
        # 1. 數據基本信息
        logger.info("=== 數據基本信息 ===")
        logger.info(f"總數據量: {len(df):,}")
        logger.info(f"唯一股票數: {len(df['證券代號'].unique()):,}")
        
        # 2. 特徵統計
        logger.info("\n=== 特徵統計 ===")
        
        # 分組統計特徵
        feature_groups = {
            "基本特徵": ['開盤價', '最高價', '最低價', '收盤價', '成交股數', '成交金額', '成交筆數'],
            "量能特徵": ['量比', '量增率', '量能趨勢'],
            "波動特徵": ['日內波動率', '振幅', '漲跌幅', '波動率趨勢'],
            "趨勢特徵": ['趨勢強度', '通道寬度', '通道寬度變化', '趨勢動能', '趨勢持續性'],
            "技術特徵": ['RSI', 'MACD', 'KD_差值', '均線糾結度', 'RSI_動能', 'MACD_動能', '技術綜合評分']
        }
        
        # 找出產業特徵
        industry_features = sorted([col for col in df.columns if '產業_' in col])
        if industry_features:
            feature_groups["產業特徵"] = industry_features
        
        # 統計每組特徵
        total_features = 0
        effective_features = 0
        
        for group, features in feature_groups.items():
            present = [f for f in features if f in df.columns]
            if present:
                valid_ratios = {f: df[f].notna().mean() * 100 for f in present}
                avg_ratio = sum(valid_ratios.values()) / len(valid_ratios)
                logger.info(f"\n{group}:")
                logger.info(f"- 特徵數: {len(present)}")
                logger.info(f"- 平均有效率: {avg_ratio:.1f}%")
                
                total_features += len(present)
                effective_features += sum(1 for r in valid_ratios.values() if r > 50)
        
        # 3. 總體統計
        logger.info("\n=== 總體統計 ===")
        logger.info(f"總特徵數: {total_features}")
        logger.info(f"有效特徵數 (>50%數據): {effective_features}")
        logger.info(f"特徵有效率: {(effective_features/total_features)*100:.1f}%")
        
        return True
        
    except Exception as e:
        logger.error(f"驗證過程出錯: {str(e)}")
        return False

In [32]:
if __name__ == "__main__":
    # 設定更詳細的日誌層級
    logging.basicConfig(level=logging.INFO)
    
    # 執行主程式
    success = main()
    
    if not success:
        print("程式執行失敗，請檢查日誌了解詳細錯誤信息")
    else:
        print("程式執行成功完成")

開始執行特徵生成主程序
INFO:FeatureGenerator:開始執行特徵生成主程序
INFO:__main__:資料統計:
INFO:__main__:- 可用股票數: 3691
INFO:__main__:- 產業類別數: 56
INFO:__main__:數據日期範圍: 2022-03-02 到 2025-02-05
檢查產業資料完整性...
INFO:FeatureGenerator:檢查產業資料完整性...
INFO:__main__:資料統計:
INFO:__main__:- 可用股票數: 3691
INFO:__main__:- 產業類別數: 56
INFO:__main__:數據日期範圍: 2022-03-02 到 2025-02-05
讀取股票主檔...
INFO:FeatureGenerator:讀取股票主檔...
載入 310 筆資料
INFO:FeatureGenerator:載入 310 筆資料
開始驗證日期範圍...
INFO:FeatureGenerator:開始驗證日期範圍...
資料日期範圍: 2024-09-30 到 2024-12-30
INFO:FeatureGenerator:資料日期範圍: 2024-09-30 到 2024-12-30
總天數: 91天
INFO:FeatureGenerator:總天數: 91天
特徵生成進度:   0%|          | 0/5 [00:00<?, ?it/s]INFO:__main__:技術指標平均有效率: 100.0%
INFO:__main__:股票 1101 對應的產業: 水泥工業
INFO:__main__:股票 1101 對應 1 個產業指數
INFO:__main__:特徵已儲存至: D:\Min\Python\Project\FA_Data\features\1101_水泥工業_20240930_20241230_20250207.csv
特徵生成進度:  20%|██        | 1/5 [00:00<00:02,  1.96it/s, 成功=1/5, 比例=20.0%]INFO:__main__:技術指標平均有效率: 100.0%
INFO:__main__:股票 2303 對應的產業: 電子工業
INFO:__main__:股票 2303 對應 

程式執行成功完成
